### Project 1: Energy commodity prices analysis 2022

The world has faced an unprecedented energy crisis this year. </br>
The increasing cost of electricity and gas is stressing household bills. threatening business productivity.

In this project, I would like to observe the energy commodity price changes through the last two years, </br>
think about the possible influcencing factors, and check if the two different energy commodities (electricity and gas) are dependant on each each other.


**Here are the questions:**</br>
Q 1: How much have the prices of energy commodities increased compared to 2021? </br>
Q 2: What could have been influencing factors for price changes? </br>
Q 3: Are the two different energy commoditiy prices(Electricity, Gas) correlated?

In [1]:
#Import required modules and library package
import pandas as pd
import numpy as np
import datetime as dt

import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objects as go
from plotly.subplots import make_subplots

##### 1. Data understanding and preparation

- To see the price development for the last two years, the most liquid markets for each commodity were selected </br>
  - Power: German Power market</br>
  - Gas: The Netherlands TTF natural gas market

The products I selected for this price anaylsis are day-ahead spot products for both markets.

1-1 Read datasets</br>
1-2 Explore the datasets </br>
1-3 Handle missing values </br>
1-4 Re-format datasets

In [2]:
# 1-1 read datasets for power and gas spot prices of 2022
pw_de_spot_2122_df = pd.read_csv("dataset/power_spot_2122", index_col=0)
gas_ttf_spot_2122_df = pd.read_csv("dataset/gas_spot_2122", index_col=0)

1-2 Explore the dataset

In [3]:
pw_de_spot_2122_df.head()

,time,value
0,2020-01-01,40.14
1,2020-01-02,38.11
2,2020-01-03,33.27
3,2020-01-04,35.70
4,2020-01-05,38.38


In [4]:
# check the dtypes of the df
pw_de_spot_2122_df.dtypes

time      object
value    float64
dtype: object

In [5]:
gas_ttf_spot_2122_df.head()

,time,value
0,2020-01-01,NaN
1,2020-01-02,11.775
2,2020-01-03,12.562
3,2020-01-06,12.075
4,2020-01-07,11.538


In [6]:
# check the dtypes of the df
gas_ttf_spot_2122_df.dtypes

time      object
value    float64
dtype: object

In [7]:
# check the size of datasets
print("The size of the power spot prices df for 2021-2022 is ", pw_de_spot_2122_df.shape)
print("The size of the gas spot prices df for 2021-2022 is ", gas_ttf_spot_2122_df.shape)

The size of the power spot prices df for 2021-2022 is  (1088, 2)
The size of the gas spot prices df for 2021-2022 is  (774, 2)


1-3 Handle missing values

In [8]:
# explore missing values
pw_de_spot_2122_df.isnull().sum()

time     0
value    7
dtype: int64

In [9]:
gas_ttf_spot_2122_df.isnull().sum()

time      0
value    12
dtype: int64

In [10]:
# check the rows having nan
pw_de_spot_2122_df[pw_de_spot_2122_df["value"].isnull()]

,time,value
102,2020-04-12,NaN
143,2020-05-23,NaN
358,2020-12-24,NaN
365,2020-12-31,NaN
456,2021-04-01,NaN
723,2021-12-24,NaN
730,2021-12-31,NaN


In [11]:
# check the rows having nan
gas_ttf_spot_2122_df[gas_ttf_spot_2122_df["value"].isnull()]

,time,value
0,2020-01-01,NaN
72,2020-04-10,NaN
73,2020-04-13,NaN
257,2020-12-25,NaN
262,2021-01-01,NaN
327,2021-04-02,NaN
328,2021-04-05,NaN
517,2021-12-27,NaN
518,2021-12-28,NaN
596,2022-04-15,NaN


Here, I used forward and backward fill methods as the prices recorded for consecutive days. It would be more accurate than replacing NaN by mean/median.

In [12]:
# fill the NaN values with forward, backward filling
pw_de_spot_2122_df = pw_de_spot_2122_df.ffill().bfill()
gas_ttf_spot_2122_df = gas_ttf_spot_2122_df.ffill().bfill()

In [13]:
# check the result of fillna
gas_ttf_spot_2122_df.isnull().sum()

time     0
value    0
dtype: int64

In [14]:
pw_de_spot_2122_df.isnull().sum()

time     0
value    0
dtype: int64

1-4 Reformat datasets

In [15]:
# before merging the two dataframe, change the data type of the column "time" into datetype
pw_de_spot_2122_df["time"] = pd.to_datetime(pw_de_spot_2122_df["time"])
gas_ttf_spot_2122_df["time"] = pd.to_datetime(gas_ttf_spot_2122_df["time"])

In [16]:
# merge the two data frames into one
energy_spot_2122_df = pd.merge(left = gas_ttf_spot_2122_df, right = pw_de_spot_2122_df, on ="time", how = "left", suffixes=("_gas", "_power"))

In [17]:
energy_spot_2122_df.shape

(774, 3)

In [18]:
spot_2122_df = energy_spot_2122_df[(energy_spot_2122_df["time"]>"2020-12-31")]

In [19]:
spot_2122_df.head()

,time,value_gas,value_power
262,2021-01-01,18.90,50.56
263,2021-01-04,19.95,57.29
264,2021-01-05,18.25,56.96
265,2021-01-06,17.73,73.25
266,2021-01-07,19.30,79.53


In [20]:
# Get the mean values for each type of commodity and year 
pw_21_mean = spot_2122_df[spot_2122_df["time"]<"2022-01-01"]["value_power"].mean()
pw_22_mean = spot_2122_df[spot_2122_df["time"]>"2021-12-31"]["value_power"].mean()
gas_21_mean = spot_2122_df[spot_2122_df["time"]<"2022-01-01"]["value_gas"].mean()
gas_22_mean = spot_2122_df[spot_2122_df["time"]>"2021-12-31"]["value_gas"].mean()
print("power price mean 2021 :", "{:.2f}".format(pw_21_mean))
print("power price mean 2022 :", "{:.2f}".format(pw_22_mean))
print("gas price mean 2021 :", "{:.2f}".format(gas_21_mean))
print("gas price mean 2022 :", "{:.2f}".format(gas_22_mean))

power price mean 2021 : 110.89
power price mean 2022 : 276.24
gas price mean 2021 : 47.09
gas price mean 2022 : 123.98


In [21]:
#Plot the spot prices'("value") changes for the whole period
fig1 = go.Figure()

fig1.add_trace(go.Scatter(x = spot_2122_df["time"], y = spot_2122_df["value_power"], name = "DE Power"))
fig1.add_hline(y= pw_21_mean, line_dash="dot", annotation_text="2021 power mean: 110.89", 
               line_color="green", annotation_position="bottom left")
fig1.add_hline(y= pw_22_mean, line_dash="dot", annotation_text="2022 power mean: 276.24", 
               line_color="purple", annotation_position="top right")
fig1.add_trace(go.Scatter(x = spot_2122_df["time"], y = spot_2122_df["value_gas"], name = "TTF Gas"))
fig1.add_hline(y= gas_21_mean, line_dash="dot", annotation_text="2021 gas mean: 47.09", 
               line_color="green", annotation_position="bottom left")
fig1.add_hline(y= gas_22_mean, line_dash="dot", annotation_text="2022 gas mean: 123.98", 
              line_color="purple", annotation_position="top right")

fig1.update_layout(
    autosize= False, width= 1000, height=700,
    title="DE power/TTF gas spot prices in 2021-2022 €/MWh",
    #xaxis = dict(tickformat="%b\n%Y"),
    xaxis_title="Time", yaxis_title="Spot price €")
fig1.show()

As shown on the fig 1, for both commodities, the prices have increased a lot. The mean values for both have increased almost by 2.5 times.</br>
Also, extreme fluctuations for both commodities are obserevd. 

To answer the second question, I would like to see the data points where huge fluctuations in prices were obsereved.</br>
- Oct 2021 : Due to a surge in energy demand after the lifted pandemic restrictions [1]</br>
- Dec 2021 ~ Jan 2022 : EU's dependancy on Russian gas, reduction in gas supplies from Russia [2]</br>
- Mar 2022: War between Russia and Ukraine, Agreement within EU on banning imports of Russian gas and oils [3]</br>
- Aug 2022: Ongoing war, additional fall in gas supply from Russia [4]

When we look at the graph above (fig 1), it can be seen that the prices of the two energy commodities fluctuate along with one another </br>
(mainly due to the gas prices change for the last few years). Now, I would like to quantify correlation between power and gas </br>

##### 2. Modeling
- I created a model an input dataset is cleaned, transformed and used for simple calculations to get daily returns and rolling volatility  
  At the end, the calculation results are visualized:
  1) Created a function for data cleaning (hadndling missing values, reformatting datasets)
  2) Created a function for daily returns, volatility calculation

Data cleaning

In [22]:
def clean_pricedata(
    df: pd.DataFrame, 
    datetype_col:str = "time", 
    )-> pd.DataFrame:
    """
    Args:
        df (pd.DataFrame): target dataset
        datetype_col (str): date type column name
        
    1 check missing values
    2 fortmat date type columns
      
    """
    # check the missing values: if true, fill the NaN values with forward/ backward fill method
    if df.isnull().values.any():
        df = df.ffill().bfill()
    else:
        pass
    
    # reformat the column types to datetype
    df[datetype_col] = pd.to_datetime(df[datetype_col])
   
    return df 

Daily returns, Volatility, Correlation calculation

- Logarithmic Price change
- 21 day-rolling volatility
- annualized rolling volatility
- 21 day-rolling correlation coefficient

In [23]:
def calculate_corr(
    df1: pd.DataFrame, 
    df2: pd.DataFrame, 
    first_commodity: str, 
    second_commodity: str,
    date_from: str
    ):
    """
    Fucntion to calculate daily return, volatility (rolling std), 
    and correlation between two different energy commodities
    Args:
        first_commodity (str): the name of energy commodity (e.g. Power, Gas etc)
        second_commodity (str): the name of energy commodity (e.g. Power, Gas etc)
    
    """
    # clean the two datasets
    first_commodity_df = clean_pricedata(df1)
    second_commodity_df = clean_pricedata(df2)
    
    # merge the two dfs
    energy_price_df =  pd.merge(
        left = first_commodity_df, right = second_commodity_df, on ="time", how = "inner", 
        suffixes=(f"_{first_commodity}", f"_{second_commodity}")
        )
    
    # calculate daily returns (log returns)
    energy_price_df[f"logReturn_{first_commodity}"] = np.log(
        energy_price_df[f"value_{first_commodity}"]/energy_price_df[f"value_{first_commodity}"].shift(1))*100
    energy_price_df[f"logReturn_{second_commodity}"] = np.log(
        energy_price_df[f"value_{second_commodity}"]/energy_price_df[f"value_{second_commodity}"].shift(1))*100
    
    # calculate 21 day-rolling volatility and annualized volatility
    energy_price_df[f"roll_std_{first_commodity}"] = energy_price_df[f"logReturn_{first_commodity}"].rolling(21).std()
    energy_price_df[f"roll_std_{second_commodity}"] = energy_price_df[f"logReturn_{second_commodity}"].rolling(21).std()

    energy_price_df[f"annual_roll_std_{first_commodity}"] = energy_price_df[f"roll_std_{first_commodity}"]*np.sqrt(252)
    energy_price_df[f"annual_roll_std_{second_commodity}"] = energy_price_df[f"roll_std_{second_commodity}"]*np.sqrt(252)
    
    # caculate correlation between two commodity types
    energy_price_df["corr"] = energy_price_df[
        f"annual_roll_std_{first_commodity}"].rolling(21).corr(energy_price_df[f"annual_roll_std_{second_commodity}"])
    
    # filter df by time
    energy_price_df = energy_price_df[energy_price_df["time"] > f"{date_from}"]

    # visualize the correlation values
    fig2 = go.Figure()

    fig2.add_trace(go.Scatter(x = energy_price_df["time"], y = energy_price_df["corr"], line = dict(color="purple")))
    fig2.update_layout(
        autosize= False, width= 1000, height=700,
        title="correlation between {} and {} in 2021-2022 €/MWh".format(first_commodity, second_commodity),
        xaxis_title="Time", yaxis_title="Correlation coefficient")
    
    return energy_price_df, fig2

In [24]:
energy_price_df, fig2 = calculate_corr(pw_de_spot_2122_df, gas_ttf_spot_2122_df, "power", "gas", "2020-12-31")

In [25]:
energy_price_df.head()

,time,value_power,value_gas,logReturn_power,logReturn_gas,roll_std_power,roll_std_gas,annual_roll_std_power,annual_roll_std_gas,corr
262,2021-01-01,50.56,18.90,5.529864,0.000000,19.191810,3.567676,304.660545,56.635105,0.852842
263,2021-01-04,57.29,19.95,12.496534,5.406722,19.215910,3.657026,305.043117,58.053492,0.875180
264,2021-01-05,56.96,18.25,-0.577682,-8.906406,18.265568,4.264261,289.956898,67.693037,0.624917
265,2021-01-06,73.25,17.73,25.152898,-2.890696,18.726018,4.350419,297.266322,69.060759,0.421399
266,2021-01-07,79.53,19.30,8.225606,8.484698,18.722424,4.643363,297.209267,73.711099,0.156772


In [26]:
fig2.show()

In [27]:
fig3 = go.Figure()

fig3.add_trace(go.Scatter(x = energy_price_df["time"], y = energy_price_df["logReturn_power"], name = "Daily return power"))
fig3.add_trace(go.Scatter(x = energy_price_df["time"], y = energy_price_df["logReturn_gas"], name = "Daily return gas"))
fig3.update_layout(
        autosize= False, width= 1400, height=400,
        title="Daily returns of power and gas prices in 2021-2022",
        xaxis_title="Time", yaxis_title="Daily returns")

##### 3. Evaluation

The figure (fig 2) created with the model depicts there is no clear positive nor negative correlation between power and gas price development through 2021-current.</br>
However, it is different from the expectation I got from the figure 1 above (showing the pure price developments).</br>
Looking at daily returns in fig 3, I could see some data points do not have similarity in their movement compared to another.

We easily see phrases, however, such as "Electricity prices, tied to the price of gas", "Natural gas is the driver for the European electricity price", [5]</br>
saying how correlated the prices of two commodities.

Then, what could be the reasons for this result given from my model?</br>
1. Natural gas is certainly influential in electricity prices, but there are other contributing factors, </br>
   such as seasonality, weather, availavility in other power generation sources and geopolitical situation, </br>
   which could make power price independantly move from the gas price development. 
   
2. Correlation analysis couldn't be appropriate to answer my third question as "Correlation doesn’t equal causality". </br> 
   In addition, simple correlation coefficient compares two different variables across the same data point.</br> 
   Other methods, such as autocorrelation or cross-correlation function could be more reasonable to find the relationship.


#### 4. Conclusion
In this project, I conducted a small experiment to observe the price developments of two different commodities, electricity and gas, </br>
and check the correlation in the price movements. With eyeball test, I found a similartiy in the price movements when I saw the pure price values.</br>
However, looking at the rolling correlation coefficient results, it was found that there was no clear correlation between the prices of the two commodities.</br>

It can be assumed that the relationship of two commdities cannot be fully explained with the simple correlation analysis as it involves a number of </br>
influencing factors, such as weather and geopolitical tensions across the world. Furthermore, more suitable method to deal with timeseries data should </br>
be used for the case.

Reference </br>
[1] https://www.euronews.com/my-europe/2021/10/28/why-europe-s-energy-prices-are-soaring-and-could-get-much-worse</br>
[2] https://www.statista.com/statistics/1267202/weekly-dutch-ttf-gas-futures/</br>
[3] https://www.aa.com.tr/en/world/highlights-of-global-energy-market-in-2022/2773628 </br>
[4] https://energy.ec.europa.eu/data-and-analysis/market-analysis_en</br>
[5] https://www.nytimes.com/2022/08/25/business/europe-electricity-prices.html